### Morocco NX Part 2: schools

In [64]:
# import modules
import geopandas as gpd
import pandas as pd
import os, sys, time
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), r'/Users/jobelanger/GOSTnets-master/GOSTnets'))
import GOSTnets as gn
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point
import matplotlib.pyplot as plt
import fiona

In [34]:
dataPth = r'/Users/jobelanger/GOSTnets-master/morocco/data'
outPth = r'/Users/jobelanger/GOSTnets-master/morocco/outputs'
pth = os.path.join(os.path.dirname(os.getcwd()), r'/Users/jobelanger/GOSTnets-master/morocco')

In [6]:
# read in largest subgraph of G (99.06% connectivity)
G = nx.read_gpickle(os.path.join(r'G_largest.pickle'))

In [7]:
# create standard speed dict
speedDict = {
                'residential': 30,  # kmph
                'primary': 60, # kmph
                'primary_link':55,
                'trunk': 40,
                'trunk_link':35,
                'secondary': 50, # kmph
                'secondary_link':45,
                'tertiary':40,
                'tertiary_link': 35,
                'unclassified':30, 
                'road':20,
                'crossing':20,
                'living_street':10
    
                } 

In [8]:
# convert graph network to time. use factor of 1000 to convert from km to meters
G_time = gn.convert_network_to_time(G, 
                                    distance_tag = 'length', 
                                    road_col = 'infra_type', 
                                    speed_dict = speedDict, 
                                    factor = 1000)

In [9]:
nx.info(G_time)

'Name: \nType: MultiDiGraph\nNumber of nodes: 16257\nNumber of edges: 43998\nAverage in degree:   2.7064\nAverage out degree:   2.7064'

In [57]:
# read in villages (origins)
villages = gpd.read_file(os.path.join(dataPth, 'tinghirVillagesP.csv'))

In [67]:
from shapely import geometry

villages = gpd.read_file(os.path.join(dataPth, 'tinghirVillagesP.csv'))
villages = pd.read_csv(villages)
geometry = [Point(xy) for xy in zip(villages.X, villages.Y)]

crs={'init':'epsg:4326'}
villages_df = GeoDataFrame(villages, crs=crs, geometry=geometry)
villages_df.to_file(driver='ESRI Shapefile', filename='villages.shp')

ValueError: Invalid file path or buffer object type: <class 'geopandas.geodataframe.GeoDataFrame'>

In [ ]:
villages_df.head()

In [39]:
villages = gn.pandana_snap_c(G_time, 
                           villages, 
                           source_crs='epsg:4326', 
                           target_crs='epsg:4326', 
                           add_dist_to_node_col=True)

In [42]:
??gn.core

In [40]:
villages.head()

,FID_1,PROVINCE,CODE_PRO,CERCLE,CODE_CER,COMMUNE,CODE_COM,FRACTION,CODE_FRA,DOUAR,...,Prov_Arabe,Reg_arabe,Code_COM_1,CODE_A,Shape_Leng,geometry,x,y,NN,NN_dist
0,4642,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,HADDOU ICHOU,...,None,None,01,04.401.05.01,115992.504936,POINT (846814.451 3461966.853),846814.4513,3.461967e+06,1014,3.564000e+06
1,6455,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,SIDI MOUHAMED IFROUTEN,...,None,None,01,04.401.05.01,115992.504936,POINT (851811.807 3456718.038),851811.8074,3.456718e+06,1014,3.560094e+06
2,11413,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,AIT EL FARSSI,...,None,None,01,04.401.05.01,115992.504936,POINT (850772.084 3476371.882),850772.0843,3.476372e+06,1014,3.578933e+06
3,19798,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,"TIZI N""TIFERKHINE",...,None,None,01,04.401.05.01,115992.504936,POINT (857317.902 3466819.838),857317.9017,3.466820e+06,1014,3.571221e+06
4,21798,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,AIT KHOUKHDENE,...,None,None,01,04.401.05.01,115992.504936,POINT (853808.560 3476154.693),853808.5600,3.476155e+06,1014,3.579445e+06


In [43]:
villages = gn.pandana_snap(G_time, 
                           villages, 
                           source_crs='epsg:4326', 
                           target_crs='epsg:4326', 
                           add_dist_to_node_col=True)

/Applications/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Applications/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [44]:
villages.head()

,FID_1,PROVINCE,CODE_PRO,CERCLE,CODE_CER,COMMUNE,CODE_COM,FRACTION,CODE_FRA,DOUAR,...,Com_Arabe,Cer_Arabe,Prov_Arabe,Reg_arabe,Code_COM_1,CODE_A,Shape_Leng,geometry,NN,NN_dist
0,4642,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,HADDOU ICHOU,...,None,None,None,None,01,04.401.05.01,115992.504936,POINT (846814.451 3461966.853),1014,3.564000e+06
1,6455,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,SIDI MOUHAMED IFROUTEN,...,None,None,None,None,01,04.401.05.01,115992.504936,POINT (851811.807 3456718.038),1014,3.560094e+06
2,11413,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,AIT EL FARSSI,...,None,None,None,None,01,04.401.05.01,115992.504936,POINT (850772.084 3476371.882),1014,3.578933e+06
3,19798,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,"TIZI N""TIFERKHINE",...,None,None,None,None,01,04.401.05.01,115992.504936,POINT (857317.902 3466819.838),1014,3.571221e+06
4,21798,OUARZAZATE,401,BOUMALNE DADES,401.05,AIT EL FARSI,401.05.01,OUEDICHEN,1,AIT KHOUKHDENE,...,None,None,None,None,01,04.401.05.01,115992.504936,POINT (853808.560 3476154.693),1014,3.579445e+06


In [ ]:
schools = gpd.read_file(os.path.join(dataPth, 'tinghirSchoolP.shp'))

In [ ]:
schools.crs

In [ ]:
schools = gn.pandana_snap_c(G_time, 
                           schools, 
                           source_crs='epsg:4326', 
                           target_crs='epsg:4326', 
                           add_dist_to_node_col=True)

In [82]:
G_time.get_edge_data

<bound method MultiGraph.get_edge_data of <networkx.classes.multidigraph.MultiDiGraph object at 0x1051f3090>>